In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, ".");

In [ ]:
using Images
using VideoIO: openvideo
using ImageView: imshow
using StaticArrays: SVector, SMatrix
using CoordinateTransformations
using Base.Test
using BenchmarkTools
using Interact
using Interpolations
using Unitful

usable motion from 3s to 20s

In [ ]:
import Cornercam
reload("Cornercam")

In [ ]:
target_size = (30, 50)

In [ ]:
video = VideoIO.openvideo("DSCF7139.MOV")

In [ ]:
background = Cornercam.mean_frame(video, (10u"s", 12u"s"))
save("background.png", background)
background

In [ ]:
original_corners = SVector{2, Int}[[236, 404], 
    [207, 1188], 
    [694, 1276], 
    [696, 340]]

desired_corners = SVector{2, Int}[[1, 1], 
    [0, target_size[2]-1],
    [target_size[1]-1, target_size[2]-1],
    [target_size[1]-1, 0]]

In [ ]:
H = Cornercam.rectify(desired_corners, original_corners)
@test all((H.(desired_corners) .≈ original_corners))

In [ ]:
samples = Cornercam.polar_samples(
    linspace(0, π/2, 200),
    linspace(0.01, 30, 50)
);

In [ ]:
function desaturate(px::RGB)
    g = gray(Gray(px))
    RGB(g, g, g)
end

function mark!(im::AbstractArray, center, radius::Integer=3, color=RGB(1., 0, 0))
    for dx in -radius:radius
        for dy in -radius:radius
            p = round.(Int, center .+ SVector(dx, dy))
            im[Tuple(p)...] = color
        end
    end
end

In [ ]:
frame = read(video)
im = desaturate.(frame)
color = RGB(1., 0, 0)
for p in H.(samples)
    mark!(im, p, 2, color)
    color = color .* 0.9995
end
im

In [ ]:
nsamples = 200
θs = linspace(0, π/2, nsamples)

A = Cornercam.visibility_gain(samples, θs);

In [ ]:
Gray.(A)

In [ ]:
# G = Tridiagonal(
#     zeros(size(A, 2) - 1), ones(size(A, 2)), .-ones(size(A, 2) - 1))
# @assert G == eye(size(A, 2)) - diagm(ones(size(A, 2) - 1), 1)

Ã = hcat(ones(size(A, 1)), A)

G = I - diagm(ones(size(Ã, 2) - 1), 1)
G = G[1:end-1, :]
G[1, :] = 0
σ = 0.085

c = eye(size(G, 2))
c[1, :] = 0

R = Tridiagonal((G' * G + c)) / σ^2

# R = Tridiagonal((G' * G + I) / σ^2)
# @assert R == (G' * G + I) / σ^2

# R_pad = Tridiagonal(vcat(0, diag(R, -1)),
#     vcat(0, diag(R, 0)),
#     vcat(0, diag(R, 1)))

λ² = 2.7


Σinv = Ã' * Ã / λ² + R
gain = (Σinv \ (Ã' / λ²));

In [ ]:
function sample(im, samples, H)
    itp = interpolate(imfilter(im, Kernel.gaussian(5)), BSpline(Linear()), OnGrid())
    [itp[Tuple(H(s))...] for s in samples]
end

In [ ]:
function imnormal(im)
    lb = minimum(x -> min(red(x), green(x), blue(x)), im)
    ub = maximum(x -> max(red(x), green(x), blue(x)), im)
    (im .- RGB(lb, lb, lb)) ./ (ub - lb)
end

In [ ]:
background_samples = sample(background, samples, H)

In [ ]:
seek(video, 4.0)
buf = read(video)

trace = hcat(map(1:40) do i
        for i in 1:6
            read!(video, buf)
        end
        pixels = sample(buf, samples, H)
        pixels .-= background_samples
        y = reshape(pixels, :)
        Lvx =  gain *  y
        Lv = Lvx[1]
        x = Lvx[2:end]
        x
        end...)

In [ ]:
imnormal(trace)